In [1]:
from sshtunnel import SSHTunnelForwarder 
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from dotenv import load_dotenv
import os
import pandas as pd
import plotly.express as px

In [2]:
load_dotenv()

True

Need to install MariaDB Connector/C from CS Package Repository https://mariadb.com/docs/server/connect/programming-languages/c/install/

Need to install MariaDB community server on the machine

In [3]:
def run_query(query_str):
    with SSHTunnelForwarder(
        (os.getenv("BASTION_SERVER_IP"), 22), #Remote server IP and SSH port
        ssh_username = os.getenv("SSH_USER_NAME"),
        ssh_pkey=os.getenv("SSH_PRIVATE_KEY_PATH"),
        remote_bind_address=(os.getenv("RDS_ENDPOINT"), 3306)) as server: 
            
        server.start() #start ssh sever
        # print('Server connected via SSH')
        
        #connect to MariaDB
        local_port = str(server.local_bind_port)
        database_name = 'financial'
        engine = create_engine(
            'mariadb+mariadbconnector://' 
            + os.getenv('DB_USERNAME') 
            + ':' 
            + os.getenv('DB_PASSWORD') 
            + '@127.0.0.1:'
            + local_port 
            + '/' 
            + database_name
        )

        Session = sessionmaker(bind=engine)
        session = Session()
        
        # print('Database session created')
        
        #test data retrieval
        test = session.execute(text(query_str))
        df = pd.DataFrame.from_records(test)
            
        session.close()

        return df

### List Tables

In [4]:
df = run_query('SHOW TABLES')
df

,0
0,account
1,card
2,client
3,disp
4,district
5,loan
6,order
7,trans


### Account 

In [17]:
df = run_query('SHOW COLUMNS FROM account')
df

,0,1,2,3,4,5
0,account_id,int(11),NO,PRI,0,
1,district_id,int(11),NO,MUL,0,
2,frequency,varchar(18),NO,,None,
3,date,date,NO,,None,


In [18]:
df = run_query('SELECT * FROM account LIMIT 5')
df

,0,1,2,3
0,1,18,POPLATEK MESICNE,1995-03-24
1,2,1,POPLATEK MESICNE,1993-02-26
2,3,5,POPLATEK MESICNE,1997-07-07
3,4,12,POPLATEK MESICNE,1996-02-21
4,5,15,POPLATEK MESICNE,1997-05-30


In [19]:
df = run_query('SELECT DISTINCT frequency FROM account')
df

,0
0,POPLATEK MESICNE
1,POPLATEK TYDNE
2,POPLATEK PO OBRATU


The data is in Czech.
- POPLATEK MESICNE: Monthly Issuance
- POPLATEK TYDNE: Weekly Issuance
- POPLATEK PO OBRATU: Issuance After Transaction

In [20]:
df = run_query('SELECT MIN(date) earliest_date, MAX(date) latest_date FROM account')
df

,0,1
0,1993-01-01,1997-12-29


In [4]:
df = run_query('SELECT district_id, COUNT(*) count FROM account GROUP BY district_id')
df.columns = ['district_id', 'count']
df

,district_id,count
0,1,554
1,2,42
2,3,50
3,4,48
4,5,65
...,...,...
72,73,56
73,74,135
74,75,51
75,76,55


In [6]:
fig = px.histogram(df, x='district_id', y='count')
fig.show()

Most of the account is from district 0 to 9

In [7]:
df = run_query('SELECT frequency, COUNT(*) count FROM account GROUP BY frequency')
df.columns = ['frequency', 'count']
fig = px.histogram(df, x='frequency', y='count')
fig.show()

Most of the account's frequency is POPLATEK MESICNE(MONTHLY FEE)

In [4]:
df = run_query("SELECT month, COUNT(*) FROM (SELECT CAST(DATE_FORMAT(date, '%Y-%m-01') AS DATETIME) month FROM account) tb GROUP BY month")
df.columns = ['month', 'count']

In [5]:
fig = px.bar(df, x='month', y='count')
fig.show()

Most of the account is created in 1993 and 1996

### Card

In [10]:
df = run_query('SHOW COLUMNS FROM card')
df

,0,1,2,3,4,5
0,card_id,int(11),NO,PRI,0,
1,disp_id,int(11),NO,MUL,None,
2,type,varchar(7),NO,,None,
3,issued,date,NO,,None,


In [12]:
df = run_query('SELECT type, COUNT(*) count FROM card GROUP BY type')
df.columns = ['type', 'count']
fig = px.histogram(df, x='type', y='count')
fig.show()

Most of the cards are Classic type.

In [13]:
df = run_query("SELECT issued_year, COUNT(*) FROM (SELECT CAST(DATE_FORMAT(issued, '%Y-01-01') AS DATETIME) issued_year FROM card) tb GROUP BY issued_year")
df.columns = ['issued_year', 'count']
fig = px.bar(df, x='issued_year', y='count')
fig.show()

Most of the cards are issued in 1998

### Client

In [4]:
df = run_query('SHOW COLUMNS FROM client')
df

,0,1,2,3,4,5
0,client_id,int(11),NO,PRI,None,
1,gender,varchar(1),NO,,None,
2,birth_date,date,NO,,None,
3,district_id,int(11),NO,MUL,None,


In [6]:
df = run_query('SELECT gender, COUNT(*) count FROM client GROUP BY gender')
df.columns = ['gender', 'count']
fig = px.histogram(df, x='gender', y='count')
fig.show()

The clients are equally distributed between males and females

In [5]:
df = run_query("SELECT birth_year, COUNT(*) FROM (SELECT CAST(DATE_FORMAT(birth_date, '%Y-01-01') AS DATETIME) birth_year FROM client) tb GROUP BY birth_year")
df.columns = ['birth_year', 'count']
fig = px.bar(df, x='birth_year', y='count')
fig.show()

Most of the clients' birth year are between 1939 to 1980

In [7]:
df = run_query("SELECT gender, birth_year, COUNT(*) FROM (SELECT gender, CAST(DATE_FORMAT(birth_date, '%Y-01-01') AS DATETIME) birth_year FROM client) tb GROUP BY gender, birth_year")
df.columns = ['gender', 'birth_year', 'count']
fig = px.bar(df, x='birth_year', y='count', color='gender')
fig.show()

The genders are quite equally distributed across birth years.

In [6]:
df = run_query('SELECT district_id, COUNT(*) count FROM client GROUP BY district_id')
df.columns = ['district_id', 'count']
fig = px.histogram(df, x='district_id', y='count')
fig.show()

Most of the clients are from district 0 to 9.

In [9]:
df = run_query("SELECT district_id, gender, COUNT(*) FROM client GROUP BY district_id, gender")
df.columns = ['district_id', 'gender', 'count']
fig = px.bar(df, x='district_id', y='count', color='gender')
fig.show()

The genders are equally distributed across districts.

### Disposition

In [14]:
df = run_query('SHOW COLUMNS FROM disp')
df

,0,1,2,3,4,5
0,disp_id,int(11),NO,PRI,None,
1,client_id,int(11),NO,MUL,None,
2,account_id,int(11),NO,MUL,None,
3,type,varchar(9),NO,,None,


In [15]:
df = run_query("SELECT type, COUNT(*) FROM disp GROUP BY type")
df.columns = ['type', 'count']
fig = px.bar(df, x='type', y='count')
fig.show()

Most of the dispositions are owner.

### District

In [16]:
df = run_query('SHOW COLUMNS FROM district')
df

,0,1,2,3,4,5
0,district_id,int(11),NO,PRI,0,
1,A2,varchar(19),NO,,None,
2,A3,varchar(15),NO,,None,
3,A4,int(11),NO,,None,
4,A5,int(11),NO,,None,
5,A6,int(11),NO,,None,
6,A7,int(11),NO,,None,
7,A8,int(11),NO,,None,
8,A9,int(11),NO,,None,
9,A10,"decimal(4,1)",NO,,None,


In [17]:
df = run_query('SELECT * FROM district LIMIT 5')
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.6,1.85,132,2159,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.9,2.21,111,2824,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.6,5.05,109,5244,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.8,4.43,118,2616,3040


### Loan

In [18]:
df = run_query('SHOW COLUMNS FROM loan')
df

,0,1,2,3,4,5
0,loan_id,int(11),NO,PRI,0,
1,account_id,int(11),NO,MUL,None,
2,date,date,NO,,None,
3,amount,int(11),NO,,None,
4,duration,int(11),NO,,None,
5,payments,"decimal(6,2)",NO,,None,
6,status,varchar(1),NO,,None,


In [20]:
df = run_query("SELECT year, COUNT(*) FROM (SELECT CAST(DATE_FORMAT(date, '%Y-01-01') AS DATETIME) year FROM loan) tb GROUP BY year")
df.columns = ['year', 'count']
fig = px.bar(df, x='year', y='count')
fig.show()

Most of the loans are granted in 1997

In [22]:
df = run_query("SELECT month_name, COUNT(*) FROM (SELECT MONTHNAME(date) month_name FROM loan) tb GROUP BY month_name")
df.columns = ['month_name', 'count']
fig = px.bar(df, x='month_name', y='count')
fig.show()

The loan grants are equally distributed across the months.

In [4]:
df = run_query('SELECT MIN(amount), MAX(amount) FROM loan')
df.columns = ['min_amount', 'max_amount']
df

,min_amount,max_amount
0,4980,590820


In [6]:
df = run_query('SELECT count(*) FROM loan')
df.columns = ['loan_count']
df

,loan_count
0,682


In [4]:
df = run_query('SELECT ROUND(amount, -4), count(*) FROM loan GROUP BY ROUND(amount, -4)')
df.columns = ['amount', 'count']
px.histogram(df, x='amount', y='count')

Most of the loan amounts are between 0 to 90K

In [7]:
df = run_query('SELECT MIN(duration), MAX(duration) FROM loan')
df.columns = ['min_duration', 'max_duration']
df

,min_duration,max_duration
0,12,60


In [8]:
df = run_query('SELECT duration, count(*) FROM loan GROUP BY duration')
df.columns = ['duration', 'count']
px.histogram(df, x='duration', y='count')

Most of the loan durations are between 20 to 39

In [9]:
df = run_query('SELECT MIN(payments), MAX(payments) FROM loan')
df.columns = ['min_payments', 'max_payments']
df

,min_payments,max_payments
0,304.00,9910.00


In [10]:
df = run_query('SELECT ROUND(payments, -2), count(*) FROM loan GROUP BY ROUND(payments, -2)')
df.columns = ['payments', 'count']
px.histogram(df, x='payments', y='count')

Most of the loan payments are between 3000 to 3900

In [12]:
df = run_query("SELECT status, COUNT(*) FROM loan tb GROUP BY status")
df.columns = ['status', 'count']
fig = px.bar(df, x='status', y='count')
fig.show()

Most of the loans are running contract, OK thus-far.

Status definition:
- 'A' stands for contract finished, no problems
- 'B' stands for contract finished, loan not paid
- 'C' stands for running contract, OK thus-far
- 'D' stands for running contract, client in debt

In [13]:
df = run_query('SELECT status, ROUND(amount, -4), count(*) FROM loan GROUP BY status, ROUND(amount, -4)')
df.columns = ['status', 'amount', 'count']
px.histogram(df, x='amount', y='count', color='status')

Most of the loans with contract finished and no problems are with lower amount. Most of the loans that contract finished and not paid have the amount between 50 - 90K.

In [14]:
df = run_query('SELECT status, duration, count(*) FROM loan GROUP BY status, duration')
df.columns = ['status', 'duration', 'count']
px.histogram(df, x='duration', y='count', color='status')

Most of the loans that contract finished and no problems have the duration less than 40. The loans with above 40 duration are running contract and OK thus-far.

In [15]:
df = run_query('SELECT status, ROUND(payments, -2), count(*) FROM loan GROUP BY status, ROUND(payments, -2)')
df.columns = ['status', 'payments', 'count']
px.histogram(df, x='payments', y='count', color='status')

The contract statuses are quite equally distributed across payments